In [1]:
import os
from dotenv import load_dotenv
import openai


load_dotenv()
print(os.environ.get("OPENAI_API_HOST"))

# openai.api_base = os.environ.get("OPENAI_API_HOST")
#openai.api_base = os.environ.get("OPENAI_API_HOST") + '/v1/chat'
#openai.api_base = "https://api.openai.com/v1"
openai.api_key = os.environ.get("OPENAI_API_KEY")

# text-embedding-ada-002 most likely
embedding_model = os.getenv("EMBED_MODEL", "text-embedding-ada-002")
text_model = os.getenv("TEXT_MODEL", "text-davinci-003")

https://chatgpt.snaplearn.ai/


In [3]:
# # We like to wrap our LLM calls to langchain models, to have a more generic interface
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.text_splitter import TokenTextSplitter
# from langchain.llms import OpenAI

# # We're using the new name llama_index, but you can find lots of example with the old name
# # gpt_index as well
# from llama_index import download_loader
# from llama_index.node_parser import SimpleNodeParser
# from llama_index import LLMPredictor, ServiceContext, PromptHelper, LangchainEmbedding


# chunk_len = 256
# chunk_overlap = 32

# embed_model = LangchainEmbedding(OpenAIEmbeddings(model=embedding_model))
# llm = OpenAI(model_name=text_model, max_tokens=2000, temperature=0)
# llm_predictor = LLMPredictor(llm=llm)
# # Llama-index parameterization
# splitter = TokenTextSplitter(chunk_size=chunk_len, chunk_overlap=chunk_overlap)
# node_parser = SimpleNodeParser(
#     text_splitter=splitter, include_extra_info=True, include_prev_next_rel=False
# )
# prompt_helper = PromptHelper.from_llm_predictor(
#     llm_predictor=llm_predictor,
# )
# service_context = ServiceContext.from_defaults(
#     llm_predictor=llm_predictor,
#     prompt_helper=prompt_helper,
#     embed_model=embed_model,
#     node_parser=node_parser,
#     # chunk_size_limit=chunk_len,
# )

In [5]:
import tiktoken
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex
from llama_index import OpenAIEmbedding, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

chunk_len = 256
chunk_overlap = 32

enc = tiktoken.get_encoding("gpt2")
tokenizer = lambda text: enc.encode(text, allowed_special={"<|endoftext|>"})
embed_model = OpenAIEmbedding()
embed_model._tokenizer = tokenizer
node_parser = SimpleNodeParser(text_splitter=TokenTextSplitter(tokenizer=tokenizer,chunk_size=chunk_len, chunk_overlap=chunk_overlap))
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, node_parser=node_parser
)

documents = SimpleDirectoryReader('/home/jinzy/work/pcg/chatdocs', recursive=True, required_exts=['.md', '.py']).load_data()
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [6]:
len(documents)

1335

In [7]:
query_engine = index.as_query_engine()
query_engine.query('which project is about chatting with multiple documnts?')

Response(response='\nThere is no project specifically about chatting with multiple documents. However, it is possible to upload multiple documents and have a conversation with them using the ChatFiles project.', source_nodes=[NodeWithScore(node=Node(text='\n\nChatFiles\n', doc_id='d96f82a7-87a7-40af-911d-7c967b15b384', embedding=None, doc_hash='ffd6d21fa449ba78f0cf92228f7d69f88fa449ceb139cf142be603b9b4bf86ef', extra_info=None, node_info={'start': 0, 'end': 12}, relationships={<DocumentRelationship.SOURCE: '1'>: '96b0593e-7b80-4def-8cb1-d60b1e73c9d3'}), score=0.8276551474399388), NodeWithScore(node=Node(text='\n\nchat with file\n1. upload a file.\n2. have a conversation with it.\n\n', doc_id='15dab744-9171-4133-8e95-b282d5e489a6', embedding=None, doc_hash='6741cb94b7aea16b8c4bc7b58da69511ad0d23cc444786b4bbaf7afcdeefcaa5', extra_info=None, node_info={'start': 0, 'end': 68}, relationships={<DocumentRelationship.SOURCE: '1'>: 'd895c924-5c2c-4b6f-b2ab-4877cd0c49ec'}), score=0.82358793179759

In [9]:
index.storage_context.persist(persist_dir="chatwithdocs")

In [10]:
query_engine.query('please tell me how the chatfiles project works? what is its main modules and what libraries it uses?')

Response(response='\nThe ChatFiles project is a repository that allows users to upload a file and have a conversation with it. It is based on the jerryjliu/llama_index library and is inspired by the mckaywrigley/chatbot-ui and madawei2699/myGPTReader libraries. The main modules of the project include a file uploader, a conversation engine, and a user interface.', source_nodes=[NodeWithScore(node=Node(text='\n\nChatFiles\n', doc_id='d96f82a7-87a7-40af-911d-7c967b15b384', embedding=None, doc_hash='ffd6d21fa449ba78f0cf92228f7d69f88fa449ceb139cf142be603b9b4bf86ef', extra_info=None, node_info={'start': 0, 'end': 12}, relationships={<DocumentRelationship.SOURCE: '1'>: '96b0593e-7b80-4def-8cb1-d60b1e73c9d3'}), score=0.8301731240272862), NodeWithScore(node=Node(text='\n\nChatFiles\n\nEN | 中文文档\n\n> this repository use jerryjliu/llama_index, based on mckaywrigley/chatbot-ui, inspired by madawei2699/myGPTReader\n\n!Chatfiles\n\n**Upload your file and have a conversation with it.**\n\n', doc_id='